In [1]:
import torch
from transformers import AutoTokenizer, EsmForMaskedLM
from transformers.models.esm.openfold_utils.protein import to_pdb, Protein as OFProtein
from transformers.models.esm.openfold_utils.feats import atom14_to_atom37
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from pyJoules.device.nvidia_device import NvidiaGPUDomain
from pyJoules.handler.pandas_handler import PandasHandler
from pyJoules.energy_meter import measure_energy

from Bio import SeqIO
import pandas as pd
import numpy as np
from time import perf_counter_ns
import matplotlib.pyplot as plt
import gc

from thop import profile, clever_format


SEQ_MAX_LEN = 5000
SAMPLES = 8000
batch_size = 1

loss_fn = nn.CrossEntropyLoss()
pandas_handler = PandasHandler()

In [2]:
ESM_8M = "facebook/esm2_t6_8M_UR50D"
ESM_35M = "facebook/esm2_t12_35M_UR50D"
ESM_150M = "facebook/esm2_t30_150M_UR50D"
ESM_650M = "facebook/esm2_t33_650M_UR50D"
ESM_3B = "facebook/esm2_t36_3B_UR50D"

In [4]:
MODEL = ESM_8M
TOKENIZER = ESM_8M

### CUDA/Torch GPU Setup

In [5]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("Using CPU")

Using GPU: NVIDIA GeForce RTX 3050 Ti Laptop GPU


## Convert FASTA file to pd.DataFrame for easier processing

In [6]:
def fasta_to_dataframe(fasta_file):
    records = []
    for seq_record in SeqIO.parse(fasta_file, 'fasta'):
        if len(records) >= SAMPLES:
            break
        # Extract the ID after '>tr'
        id_info = seq_record.id.split('|')[1] # Adjust the index based on your FASTA file format
        sequence = str(seq_record.seq)
        if (len(sequence) >= SEQ_MAX_LEN):
            continue 
        
        # Generate a random index within the range of the sequence length
        random_index = np.random.randint(0, min(len(sequence), SEQ_MAX_LEN))
        # Replace the character at the random index with '<mask>'
        masked_sequence = sequence[:random_index] + '<mask>' + sequence[random_index+1:]
        
        records.append([id_info, sequence, masked_sequence])
    
    # Create a DataFrame from the records
    df = pd.DataFrame(records, columns=['ID', 'Sequence', 'Masked_Sequence'])
    return df

## Create Dataset

In [7]:
fasta_df = fasta_to_dataframe("human_protein_seq/uniprotkb_proteome_UP000005640.fasta")


In [8]:
fasta_df.head()

,ID,Sequence,Masked_Sequence
0,A0A075B6G3,MLWWEEVEDCYEREDVQKKTFTKWVNAQFSKFGKQHIENLFSDLQD...,MLWWEEVEDCYEREDVQKKTFTKWVNAQFSKFGKQHIENLFSDLQD...
1,A0A087WV00,MDAAGRGCHLLPLPAARGPARAPAAAAAAAASPPGPCSGAACAPSA...,MDAAGRGCHLLPLPAARGPARAPAAAAAAAASPPGPCSGAACAPSA...
2,A0A087WZT3,MELSAEYLREKLQRDLEAEHVLPSPGGVGQVRGETAASETQLGS,MELSAEYLREKLQRDLEAEHV<mask>PSPGGVGQVRGETAASETQLGS
3,A0A087X1C5,MGLEALVPLAMIVAIFLLLVDLMHRHQRWAARYPPGPLPLPGLGNL...,MGLEALVPLAMIVAI<mask>LLLVDLMHRHQRWAARYPPGPLPLP...
4,A0A087X296,MSRSLLLWFLLFLLLLPPLPVLLADPGAPTPVNPCCYYPCQHQGIC...,MSRSLLLWFLLFLLLLPPLPVLLADPGAPTPVNPCCYYPCQHQGIC...


In [9]:
fasta_df.shape

(8000, 3)

## Preparing your model and tokenizer

Now we load our model and tokenizer. If using GPU, use `model.cuda()` to transfer the model to GPU.

In [10]:
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER)
model = EsmForMaskedLM.from_pretrained(MODEL)

In [67]:
class FastaDataset(Dataset):
    def __init__(self, fasta_df, tokenizer):
        self.fasta_df = fasta_df
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.fasta_df)

    def __getitem__(self, idx):
        fasta_row = self.fasta_df.iloc[idx]
        latency_time_stamps = []
        # TIME LOG: start
        latency_time_stamps.append(perf_counter_ns())

        # Tokenize the input and move it to the correct device
        inputs = tokenizer(fasta_row["Masked_Sequence"], max_length=min(len(fasta_row["Masked_Sequence"]), SEQ_MAX_LEN), truncation=True, return_tensors='pt')
        
        # TIME LOG: tokenize
        latency_time_stamps.append(perf_counter_ns())
        
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        # TIME LOG: move_to_gpu
        latency_time_stamps.append(perf_counter_ns())
        
        unmasked_text = fasta_row['Sequence']
        masked_text = fasta_row["Masked_Sequence"]
        mask_position = masked_text.find('<mask>')
        actual_token = unmasked_text[mask_position]
        
        with torch.no_grad():
            logits = model(**inputs).logits
        
        # TIME LOG: model
        latency_time_stamps.append(perf_counter_ns())

        memusage = 0
        params = 0

        with torch.no_grad():
            profiler_input = inputs['input_ids']
            macs, p = profile(model, inputs=(profiler_input, ))
            memusage = macs 
            params = p

        # Convert the actual token to its corresponding ID
        actual_token_id = tokenizer.encode(actual_token, add_special_tokens=False)[0]
        one_hot = nn.functional.one_hot(torch.tensor([actual_token_id]), 33)[0].float()
        one_hot = one_hot.to(device)
        # Calculate the cross-entropy loss
        logits = logits[0][mask_position]
        loss = loss_fn(logits, one_hot)
        
        # Convert the fasta_row to a dictionary before returning
        fasta_row_dict = fasta_row.to_dict()
        
        return loss, latency_time_stamps, fasta_row_dict, memusage, params


In [68]:
# Instantiate the dataset
fasta_dataset = FastaDataset(fasta_df, tokenizer)

# Create a DataLoader
dataloader = DataLoader(fasta_dataset, batch_size=batch_size, shuffle=False)

In [69]:
# Ensure the model is in evaluation mode
model.eval()

model.to(device)

EsmForMaskedLM(
  (esm): EsmModel(
    (embeddings): EsmEmbeddings(
      (word_embeddings): Embedding(33, 320, padding_idx=1)
      (dropout): Dropout(p=0.0, inplace=False)
      (position_embeddings): Embedding(1026, 320, padding_idx=1)
    )
    (encoder): EsmEncoder(
      (layer): ModuleList(
        (0-5): 6 x EsmLayer(
          (attention): EsmAttention(
            (self): EsmSelfAttention(
              (query): Linear(in_features=320, out_features=320, bias=True)
              (key): Linear(in_features=320, out_features=320, bias=True)
              (value): Linear(in_features=320, out_features=320, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
              (rotary_embeddings): RotaryEmbedding()
            )
            (output): EsmSelfOutput(
              (dense): Linear(in_features=320, out_features=320, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (LayerNorm): LayerNorm((320,), eps=1e-05, elementwise_a

In [70]:
print(f"Running test on {fasta_df.shape[0]} sequences.")

gc.disable()

# set up accuracy & latency info - leave this in the same code block so that it resets each time
accuracy = []
latency = pd.DataFrame(columns=['start', 'tokenize', 'move_to_gpu', 'model', "end"])
sequence_length = []
macs = []
memory_usage = []
ctr = 0
iter_dl = iter(dataloader)

@measure_energy(domains=[NvidiaGPUDomain(0)], handler=pandas_handler)
def run(accuracy, latency, iter_dl):
    loss, latency_time_stamps, fasta_row, mac, params = next(iter_dl)

    fasta_df = pd.DataFrame.from_dict(fasta_row)

    # TIME LOG: end
    latency_time_stamps.append(perf_counter_ns())

    # Append the loss to a list or use it as needed
    accuracy.append(loss.item())
    
    macs.append(mac.item())
    memory_usage.append(params.item())    
    # memory_usage.append(torch.cuda.memory_allocated())

    # unrolled loop
    latency_time_stamps[0] = int(latency_time_stamps[0])
    latency_time_stamps[1] = int(latency_time_stamps[1])
    latency_time_stamps[2] = int(latency_time_stamps[2])
    latency_time_stamps[3] = int(latency_time_stamps[3])
    
    # print(latency_time_stamps)
    latency.loc[len(latency.index)] = latency_time_stamps
    # print(fasta_df)
    
    return len(fasta_df["Sequence"][0])

torch.cuda.empty_cache()

for ctr in range(5):
# for ctr in range(fasta_df.shape[0]):
    if ctr % 100 == 0:
        percentage = "{:.2f}".format(ctr * 100 / fasta_df.shape[0])
        print(f"Processing... {percentage}%")

    torch.cuda.empty_cache()
    # try to save as much memory as possible by reducing fragmentation
    gc.collect()
    
    # load model to GPU
    model.to(device)
    sequence_length.append(run(accuracy, latency, iter_dl))
    
    # try to remove anything no longer needed on the GPU mem
    gc.collect()
            
gc.enable()
gpu_util_df = pandas_handler.get_dataframe()

Running test on 8000 sequences.
Processing... 0.00%
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.normalization.LayerNorm'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.normalization.LayerNorm'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.normalization.LayerNorm'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register count_normalization() for <class 'torch.nn

### Cross Entropy Loss x Sequence Length

In [ ]:
accuracy = np.array(accuracy) 
sequence_length = np.array(sequence_length)
flops = np.array(macs)
memory_usage = np.array(memory_usage)

latency_df = pd.DataFrame(columns=['tokenization', 'move_inputs_to_gpu', 'model_process', 'cross_entropy_loss'])

latency_df["tokenization"] = latency["tokenize"] - latency["start"]
latency_df["move_inputs_to_gpu"] = latency["move_to_gpu"] - latency["tokenize"]
latency_df["model_process"] = latency["model"] - latency["move_to_gpu"]
latency_df["cross_entropy_loss"] = latency["end"] - latency["model"]

duration = gpu_util_df["duration"] * 100000

lat_gpu_slen = {"Latency": duration, "GPU_Utilization": gpu_util_df["nvidia_gpu_0"], "Sequence_Length": sequence_length}

In [ ]:
mpath = MODEL.split('/')[1]

np.save(mpath + 'macs', flops)
np.save(mpath + 'memory', memory_usage)
np.save(mpath + 'accuracy', accuracy)
np.save(mpath + 'seql', sequence_length)
latency_df.to_csv("csv/" + mpath + '_latency')
gpu_util_df.to_csv("csv/" + mpath + '_gpu_slen')

In [ ]:
# Create a scatter plot
plt.scatter(sequence_length, accuracy)

# Label the axes
plt.xlabel('Sequence Length')
plt.ylabel('Cross Entropy Loss')

# Add a title
plt.title('Sequence Length vs. Cross Entropy Loss')

# Display the plot
plt.show()

To get a better visualization, limit it to sequences below 10k length

In [ ]:
reduced_s_len = []
reduced_acc = []

# remove outliers
for id, s_len in enumerate(sequence_length):
    if s_len <= 10000:
        reduced_s_len.append(sequence_length[id])
        reduced_acc.append(accuracy[id])

# Create a scatter plot
plt.scatter(reduced_s_len, reduced_acc)

# Label the axes
plt.xlabel('Sequence Length')
plt.ylabel('Cross Entropy Loss')

# Add a title
plt.title('[Truncated] Sequence Length vs. Cross Entropy Loss')

# Display the plot
plt.show()

## Latency/Time Log

In [ ]:
# latency = pd.DataFrame(columns=['start', 'tokenize', 'move_to_gpu', 'model', "end"])
# get_cel = get cross entropy loss
latency_df = pd.DataFrame(columns=['tokenization', 'move_inputs_to_gpu', 'model_process', 'cross_entropy_loss'])

latency_df["tokenization"] = latency["tokenize"] - latency["start"]
latency_df["move_inputs_to_gpu"] = latency["move_to_gpu"] - latency["tokenize"]
latency_df["model_process"] = latency["model"] - latency["move_to_gpu"]
latency_df["cross_entropy_loss"] = latency["end"] - latency["model"]

latency_sums = latency_df.sum()
# Create a bar plot
latency_sums.plot(kind='bar')

# Add title and labels
plt.title('Total Latency per Step')

# Display the plot
plt.show()

### Latency & GPU Usage vs Sequence Length

In [ ]:
gpu_util_df.head()

In [ ]:
# Normalize Duration
duration = gpu_util_df["duration"] * 100000

lat_gpu_slen = {"Latency": duration, "GPU_Utilization": gpu_util_df["nvidia_gpu_0"], "Sequence_Length": sequence_length}
lat_gpu_slen_df = pd.DataFrame(lat_gpu_slen)
lat_gpu_slen_df = lat_gpu_slen_df.sort_values(by = 'Sequence_Length')

# Plot multiple lines
plt.plot(lat_gpu_slen_df["Sequence_Length"], lat_gpu_slen_df["GPU_Utilization"], label='GPU_Utilization')
plt.plot(lat_gpu_slen_df["Sequence_Length"], lat_gpu_slen_df["Latency"], label='Latency')

plt.xlabel('Sequence Length')
plt.ylabel('10ns / Joule')

plt.title("Latency/GPU Util vs Sequence Length")
plt.legend()

plt.show()

Remove Outliers by limiting it to 10k

In [ ]:
reduced_lat_gpu_slen_df = lat_gpu_slen_df[lat_gpu_slen_df["Sequence_Length"] <= 10000]

# Plot multiple lines
plt.plot(reduced_lat_gpu_slen_df["Sequence_Length"], reduced_lat_gpu_slen_df["GPU_Utilization"], label='GPU_Utilization')
plt.plot(reduced_lat_gpu_slen_df["Sequence_Length"], reduced_lat_gpu_slen_df["Latency"], label='Latency')

plt.xlabel('Sequence Length')
plt.ylabel('10ns / Joule')

plt.title("[Truncated] Latency/GPU Util vs Sequence Length")
plt.legend()

plt.show()